### База данных "Архив погоды"

1. Создание парсера для скачивания данных о погоде по метеостанциям
2. Создание CSV-файлов данных по погоде
3. Объединение файлов в единую базу данных по погоде
4. Работа с ежемесячным файлом

#### Load necessary library

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json
import io
import certifi
import urllib
import urllib.request as req

#### Web-парсер для обработки данных с сайта www.pr5.ru

In [2]:
def load_weather (wd, URL, START_DATE, END_DATE, forma, codir):
    try:
        # запускаем selenium, выставляем неявное ожидание и переходим на страничку с архивом
        wd.implicitly_wait(15)
        wd.get(URL)

        # здесь и далее для указания нужных элементов страницы 
        # используются XPath 
        # выделить нужный элемент -> Посмотреть код -> Copy -> Copy XPath 
        # переходим по вкладке «Архив погоды» 
        d_vkl = wd.find_element_by_xpath('//*[@id="tabSynopDLoad"]')
        d_vkl.click()

        # заполняем начальную и конечную даты 
        start_date_input = wd.find_element_by_xpath('//*[@id="calender_dload"]')
        end_date_input = wd.find_element_by_xpath('//*[@id="calender_dload2"]')

        start_date_input.clear()
        start_date_input.send_keys(START_DATE)

        end_date_input.clear()
        end_date_input.send_keys(END_DATE)

        # выбираем формат файла
        if forma == 'csv':
            # формат файла 'csv'
            radio_csv = wd.find_element_by_xpath('//*[@id="toFileMenu"]/form/table[2]/tbody/tr[2]/td[3]/label/span')
            radio_csv.click()
            # выбираем кодировку файла
            if codir == 'utf-8':
                #кодировка (utf-8)
                radio_utf = wd.find_element_by_xpath('//*[@id="toFileMenu"]/form/table[2]/tbody/tr[3]/td[3]/label')
            elif codir == 'ansi':
                #кодировка (ansi)
                radio_utf = wd.find_element_by_xpath('//*[@id="toFileMenu"]/form/table[2]/tbody/tr[3]/td[2]/label')
            else:
                #кодировка (unicode)
                radio_utf = wd.find_element_by_xpath('//*[@id="toFileMenu"]/form/table[2]/tbody/tr[3]/td[4]/label')
            radio_utf.click()
        else:
            # формат файла 'excel'
            radio_csv = wd.find_element_by_xpath('//*[@id="toFileMenu"]/form/table[2]/tbody/tr[2]/td[2]/label/span')
            radio_csv.click()
        
        # нажимаем кнопку скачивания, чеез некоторое время появится ссылка на файл с архивом погоды
        download_button = wd.\
            find_element_by_xpath('//*[@id="toFileMenu"]/form/table[2]/tbody/tr[3]/td[6]/table/tbody/tr/td[1]/div/div')
        download_button.click()

        # скачиваем файл
        try:
            download_link = wd.find_element_by_xpath('//*[@id="f_result"]/a')
            download_link_href = download_link.get_attribute("href")
            wd.get(download_link_href)
            txt = 'Файл загружен'
        except:
            txt = 'Файл не был загружен'
        # возврат результатов (link и результат загрузки)
    except:
        txt = 'Архив отсутствует'
    return URL, txt

#### Создание входных данных

In [3]:
# Создание списка с ID метеостанций
l = pd.read_excel("DATA/Список метеостанций и описание показателей.xlsx", sheet_name='USSR', header=[0])
links_rp5 = l['Link'].tolist()

In [4]:
# Создание списка с ID метеостанций
l = pd.read_csv("DATA/Meteo_link_full.csv", sep=';', header=[0])
links_rp5 = l['Link'].values.tolist()

# Создание списка с ID метеостанций (пустые)
'''l = pd.read_csv("Meteo_link_empty.csv", header=[0])
links_rp5 = l['Link'].values.tolist()'''

'l = pd.read_csv("Meteo_link_empty.csv", header=[0])\nlinks_rp5 = l[\'Link\'].values.tolist()'

In [5]:
# создание словаря
dicts = []

#выбор формата файла
forma = 'csv'
# выбор кодировки
codir = 'utf-8'

# для селениума нужен driver web-браузера, прописываем к нему путь
DRIVER = 'C:/Users/uovorop/Downloads/EdgeDriver/msedgedriver.exe'
#DRIVER = 'C:/Users/uovorop/Downloads/ChromeDriver/chromedriver.exe'

# за какой период нужны данные
START_DATE = '01.07.2021'
END_DATE = '31.12.2021'

# запуск драйвера для браузера
wd = webdriver.Edge(DRIVER)
#wd = webdriver.Chrome(DRIVER)

SessionNotCreatedException: Message: session not created: This version of MSEdgeDriver only supports MSEdge version 99
Current browser version is 105.0.1343.33 with binary path C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe


#### Закачка Архива данных с сайта www.rp5.ru

In [ ]:
for link in links_rp5:
    dicts.append(load_weather (wd, link, START_DATE, END_DATE, forma, codir))

In [8]:
wd.close()

In [9]:
l_empty = []
for l in dicts:
    if l[1] == 'Архив отсутствует':
        l_empty.append(l[0])
print(len(l_empty))

8


In [10]:
l_failure = []
for l in dicts:
    if l[1] == 'Файл не был загружен':
        l_failure.append(l[0])
print(len(l_failure))

223


In [11]:
l_empty

['http://rp5.ru/archive.php?wmo_id=25356&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=26825&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=35067&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=37089&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=37279&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=37298&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=37959&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=38880&lang=ru']

In [12]:
l_failure

['http://rp5.ru/archive.php?wmo_id=20087&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20107&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20471&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20476&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20665&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20674&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20744&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20871&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20891&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20946&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20967&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20973&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=20982&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=21711&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=21802&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=21921&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=21931&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=22028&lang=ru',
 'http://rp5.ru/archive.php?wmo_id=22095&lang=ru',
 'http://rp5.ru/archive.php?wmo

#### Повторяемая часть кода
Повторить описанную ниже часть кода, до тех пор пока длина списка "l_recycle" не будет равна нулю

In [36]:
l_recycle = []
for l in dicts2:
    if l[1] == 'Файл не был загружен':
        l_recycle.append(l[0])
print(len(l_recycle))

1


In [37]:
l_recycle

['http://rp5.ru/archive.php?wmo_id=31005&lang=ru']

In [38]:
# создание словаря
dicts2 = []

#выбор формата файла
forma2 = 'csv'
# выбор кодировки
codir2 = 'utf-8'

# для селениума нужен driver web-браузера, прописываем к нему путь
DRIVER2 = 'C:/Users/uovorop/Downloads/EdgeDriver/msedgedriver.exe'
#DRIVER = 'C:/Users/uovorop/Downloads/ChromeDriver/chromedriver.exe'

# за какой период нужны данные
START_DATE2 = '01.07.2021'
END_DATE2 = '31.12.2021'

# запуск драйвера для браузера
wd2 = webdriver.Edge(DRIVER)
#wd = webdriver.Chrome(DRIVER)

In [39]:
#for link in l_failure:
#for link in l_recycle:
for link in l_empty:
    dicts2.append(load_weather (wd2, link, START_DATE2, END_DATE2, forma2, codir2))

In [40]:
wd2.close()

In [41]:
l_load = []
for l in dicts:
    if l[1] == 'Файл загружен':
        l_load.append(l[0])
print(len(l_load))

l_full = l_load + l_recycle
# Сохранение данных в файл CSV-формата
pd.DataFrame(l_full).to_csv('Meteo_link_full.csv', index=False)

- \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ - \ 